# Basel Precipitaion Forecast
In this project we train a prediction model for precipitation by using some data about weather.

## Abstract
- Suppose we have some data about a day and want to predict whether it will rain or not.
- First find patterns for T, H, W, P, C and also for Precipitation.
- Then train a model with different techniques to predict rainfall.
- Loading the essential libraries and then loading and reading the dataset.
- Visualizing the dataset using charts, graphs, etc., to understand the data better.
- Pre-processing the data using various data cleaning and manipulation techniques.
- Applying multiple machine learning algorithms to train the models and evaluate their performance using different metrics.

## Introduction
- Tell something about Basel city and importance of precipitation forecasting.
- Tell something about importance of up-to-date data of climate and new patterns.
- Tell something about advantages of forecasting and why it is important today.

In [24]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import sklearn

from getdata import get_data, get_daily_temperature
from getdata import get_daily_date

ImportError: cannot import name 'get_daily_date' from 'getdata' (/Users/fraxea/Desktop/up/weather/getdata.py)

## Dataset
**You can find and download the dataset in [this](https://www.meteoblue.com/en/weather/archive/export) link.**

##### About dataset
This dataset contains some attributes about weather for Basel, a city in northwest Switzerland, from January, 2014 till November, 2023 with hourly resolution.
##### Cleaning dataset
The first nine rows are some basic information about location of city and units of measurements which we do not need it. Also there are some rows at the end of dataset which are empty(missing data) and we have to filter them as well. In this project we do not consider wind direction, so we do not cosider this column.

In [ ]:
missing_data, cleaned_data = get_data()
cleaned_data.drop(cleaned_data.columns[4], inplace=True, axis=1)
print(f"Number of missing rows: {missing_data}")
print(f"Number of examples:     {cleaned_data.shape[0]}")
print(f"Number of features:     {cleaned_data.shape[1]}")
pd.concat([cleaned_data.head(3), cleaned_data.tail(3)]).style

Number of missing rows: 191
Number of examples:     86473
Number of features:     6


,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Relative Humidity [2 m],Basel Wind Speed [100 m],Basel Cloud Cover Total,Basel Mean Sea Level Pressure [MSL]
20140101T0000,0.310245,0.000000,88.457619,16.750641,52.000000,1016.200012
20140101T0100,1.110245,0.000000,84.218651,17.418196,94.000000,1016.299988
20140101T0200,2.220245,0.200000,82.354805,18.250259,94.000000,1016.599976
20231112T2200,11.230246,0.600000,89.942604,27.295977,98.000000,1012.900024
20231112T2300,11.040245,0.600000,92.824905,23.770906,99.000000,1012.900024
20231113T0000,11.380245,0.700000,91.976036,27.511158,100.000000,1013.299988


## Features
Each row represents an example and each column represents a feature. Here is the list of features:
- Temperature
- Precipitation
- Relative Humidity
- Wind speed
- Cloud cover
- Pressure

### Make examples daily
Forecasting for a whole day is more general than one hour, so we decide to merge each 24 examples to convert the resolution to daily. A good questio is how? One way is to get a mean for each feature. Specificly for temperature, having maximum, minimum, and average is good.

In [10]:
dates = get_date_daily(cleaned_data)
print(f"Number of days:         {dates.shape[0]}")

NameError: name 'get_date_daily' is not defined

## Temperature
Basel is a relatively cold city:

In [8]:
MIN_T, MAX_T, MEAN_T, mean_temperature, min_temperature, max_temperature, var_temperature = get_daily_temperature(cleaned_data)
print(f"Minimum temperature is: {MIN_T:.1f} °C")
print(f"Maximum temperature is: {MAX_T:.1f} °C")
print(f"Mean temperature is:    {MEAN_T:.1f} °C")

Minimum temperature is: -13.0 °C
Maximum temperature is: 38.6 °C
Mean temperature is:    12.1 °C
Number of days:         3604


In [ ]:
# CODE

A pattern that can be thaught is, when the difference between maximum and minimum is lower, the precipitation is higher. In other words, when the variance of temperature is relatively low, the precipitation is high. One advantage of using variance instead of maximum and minimum is you store one feature instead of two. So we model once with variance and other time with max/min.

## Models
Let's have a look at each one in deep

### Linear Model
This is like ...
If we apply it, results:
###### SOME FIGUERS AND PLOTS TO SHOW THE RESULTS
Calculate performance, error, advantages and disadvantages

### Other Models
This is like ...
If we apply it, results:
###### SOME FIGUERS AND PLOTS TO SHOW THE RESULTS
Calculate performance, error, advantages and disadvantages

## Conclusion
- Forecasting is good.
- Undestand that these features have a stronger affect in result.
- This Model is better in performance.
- Add references if necessary.